In [1]:
from utils import DataPipeline

data_pipe = DataPipeline(priors_path='../input/order_products__prior.csv',
                         train_path='../input/order_products__train.csv',
                         orders_path='../input/orders.csv',
                         product_path='../input/products.csv')

loading prior
loading train
loading orders
loading products
priors (32434489, 4): order_id, product_id, add_to_cart_order, reordered
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
train (1384617, 4): order_id, product_id, add_to_cart_order, reordered
optimizing memory usages
computing product f
add order info to priors
computing user f
user f (206209, 3)
compute userXproduct f - this is long...
to dataframe (less memory)
user X product f 13307953


In [2]:
import lightgbm as lgb

model = lgb.Booster(model_file='../models/model.txt')

In [3]:


train_orders, test_orders = data_pipe.get_test_and_train()
df_train, labels = data_pipe.create_feature_labelled_data(train_orders, labels_given=True)


split orders : train, test
build candidate list
order row 10000
order row 20000
order row 30000
order row 40000
order row 50000
order row 60000
order row 70000
order row 80000
order row 90000
order row 100000
order row 110000
order row 120000
order row 130000
user related features
order related features
product related features
user_X_product related features
order_id                              int32
product_id                            int32
user_total_orders                     int16
user_total_items                      int16
total_distinct_items                  int16
user_average_days_between_orders    float32
user_average_basket                 float32
order_hour_of_day                      int8
days_since_prior_order              float32
days_since_ratio                    float32
aisle_id                               int8
department_id                          int8
product_orders                      float32
product_reorders                    float32
product_reorder_rate  

In [4]:
import importlib
import utils

importlib.reload(utils)

df_train, labels, df_val, val_labels =\
    utils.split_train_val(df=df_train, labels=labels, val_size=0.05, train_orders=None)
# df_train, labels, df_val, val_labels, train_orders, val_orders =\
#     utils.split_train_val(df=df_train, labels=labels, val_size=0.05, train_orders=None)
    
features_to_use = ['user_total_orders', 'user_total_items', 'total_distinct_items',
                   'user_average_days_between_orders', 'user_average_basket',
                   'order_hour_of_day', 'days_since_prior_order', 'days_since_ratio',
                   'aisle_id', 'department_id', 'product_orders', 'product_reorders',
                   'product_reorder_rate', 'UP_orders', 'UP_orders_ratio',
                   'UP_average_pos_in_cart', 'UP_reorder_rate', 'UP_orders_since_last',
                   'UP_delta_hour_vs_last']

In [5]:
preds = model.predict(df_val[features_to_use], num_iteration=model.best_iteration)
df_val['pred'] = preds

# Filter by confidence
# confidence_filter = filter_by_confidence(df_test=df_val,
#                                          test_orders=val_orders,
#                                          threshold=0.22)

# Submission
print(df_val.head())

    order_id  product_id  user_total_orders  user_total_items  \
1    1187899         196                 11                59   
10   1187899       35951                 11                59   
31   1492625          79                 15               195   
47   1492625       16521                 15               195   
59   1492625       19156                 15               195   

    total_distinct_items  user_average_days_between_orders  \
1                     18                         19.000000   
10                    18                         19.000000   
31                   102                         16.285715   
47                   102                         16.285715   
59                   102                         16.285715   

    user_average_basket  order_hour_of_day  days_since_prior_order  \
1              5.363636                  8                    14.0   
10             5.363636                  8                    14.0   
31            13.000000   

In [14]:
# original_vals = df_val.groupby('order_id',as_index=False)['product_id'].aggregate(lambda x: list(x))

d = dict()
original = dict()
for row in df_val.itertuples():
    try:
        original[row.order_id].append(row.product_id)
    except:
        original[row.order_id] = [row.product_id]
    if row.pred > 0.01:
        try:
            d[row.order_id].append(row.product_id)
        except:
            d[row.order_id] = [row.product_id]

for order in df_val.order_id:
    if order not in d:
        d[order] = []

print(d[2993918])
print(original[2993918])

[41555, 23157, 5268, 27896, 17167, 28993, 41290, 5097]
[41555, 23157, 5268, 27896, 17167, 28993, 41290, 5097]


In [15]:
fp = 0
tp = 0
fn = 0

total_products = 0

for key in original:
    true = original[key]
    predicted = d[key]
    
    total_products += len(true)
    
    for entry in predicted:
        if entry in true:
            tp += 1
        elif entry not in true:
            fp += 1
    
    for entry in true:
        if entry not in predicted:
            fn += 1
            
print('TP: {}'.format(tp))
print('FP: {}'.format(fp))
print('FN: {}'.format(fn))
print('Total items: {}'.format(total_products))

precision = tp/ (tp+fp)
recall = tp / (tp+fn)
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1: {}'.format(2*(precision * recall)/(precision + recall)))


TP: 374493
FP: 0
FN: 50248
Total items: 424741
Precision: 1.0
Recall: 0.8816973167177173
F1: 0.9371298017852093


In [17]:
original_vals.head(100)

,product_id,user_total_orders,user_total_items,total_distinct_items,user_average_days_between_orders,user_average_basket,order_hour_of_day,days_since_prior_order,days_since_ratio,aisle_id,...,product_orders,product_reorders,product_reorder_rate,UP_orders,UP_orders_ratio,UP_average_pos_in_cart,UP_reorder_rate,UP_orders_since_last,UP_delta_hour_vs_last,pred
order_id,,,,,,,,,,,,,,,,,,,,,
36,"[45572, 48679, 12456, 16759]","[23, 23, 23, 23]","[187, 187, 187, 187]","[101, 101, 101, 101]","[15.681818008422852, 15.681818008422852, 15.68...","[8.1304349899292, 8.1304349899292, 8.130434989...","[18, 18, 18, 18]","[30.0, 30.0, 30.0, 30.0]","[1.91304349899292, 1.91304349899292, 1.9130434...","[53, 83, 106, 83]",...,"[148.0, 51939.0, 12391.0, 35820.0]","[53.0, 34408.0, 7680.0, 20763.0]","[0.3581081032752991, 0.6624694466590881, 0.619...","[1, 2, 1, 2]","[0.043478261679410934, 0.08695652335882187, 0....","[10.0, 3.5, 8.0, 8.5]","[0.043478261679410934, 0.08695652335882187, 0....","[2, 2, 1, 1]","[8, 8, 4, 4]","[0.0302297990989199, 0.11436151123107598, 0.07..."
96,"[34050, 2067, 13497]","[7, 7, 7]","[43, 43, 43]","[33, 33, 33]","[21.0, 21.0, 21.0]","[6.142857074737549, 6.142857074737549, 6.14285...","[20, 20, 20]","[30.0, 30.0, 30.0]","[1.4285714626312256, 1.4285714626312256, 1.428...","[31, 3, 42]",...,"[16987.0, 3138.0, 1022.0]","[12440.0, 1902.0, 573.0]","[0.7323247194290161, 0.6061185598373413, 0.560...","[1, 1, 1]","[0.1428571492433548, 0.1428571492433548, 0.142...","[6.0, 2.0, 5.0]","[0.1428571492433548, 0.1428571492433548, 0.142...","[3, 2, 5]","[8, 0, 3]","[0.07597452370179637, 0.08095663653112262, 0.0..."
98,"[19508, 12385, 16995, 14477, 26769, 9373, 9896...","[41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41]","[1234, 1234, 1234, 1234, 1234, 1234, 1234, 123...","[207, 207, 207, 207, 207, 207, 207, 207, 207, ...","[9.050000190734863, 9.050000190734863, 9.05000...","[30.09756088256836, 30.09756088256836, 30.0975...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]","[14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14....","[1.5469613075256348, 1.5469613075256348, 1.546...","[-128, 69, 96, -125, 19, 84, 106, 108, 83, 31,...",...,"[14905.0, 1143.0, 3093.0, 2280.0, 1450.0, 745....","[6681.0, 300.0, 1645.0, 913.0, 350.0, 371.0, 6...","[0.4482388496398926, 0.2624672055244446, 0.531...","[1, 1, 8, 1, 3, 9, 25, 16, 4, 38, 10, 18]","[0.024390242993831635, 0.024390242993831635, 0...","[1.0, 13.0, 13.625, 28.0, 24.33333396911621, 2...","[0.024390242993831635, 0.024390242993831635, 0...","[2, 23, 16, 9, 30, 4, 1, 1, 3, 1, 1, 3]","[3, 6, 7, 4, 8, 3, 8, 8, 4, 8, 8, 4]","[0.06891764313198238, 0.008321400935127416, 0...."
112,"[47766, 43768]","[5, 5]","[66, 66]","[43, 43]","[21.0, 21.0]","[13.199999809265137, 13.199999809265137]","[14, 14]","[26.0, 26.0]","[1.2380952835083008, 1.2380952835083008]","[24, 83]",...,"[176815.0, 23388.0]","[134044.0, 13806.0]","[0.75810307264328, 0.5903027057647705]","[2, 1]","[0.4000000059604645, 0.20000000298023224]","[4.0, 17.0]","[0.4000000059604645, 0.20000000298023224]","[2, 2]","[1, 1]","[0.3950519370841972, 0.14244103475152053]"
170,[44756],[7],[60],[33],[20.33333396911621],[8.571428298950195],[13],[14.0],[0.6885245442390442],[11],...,[152.0],[57.0],[0.375],[2],[0.2857142984867096],[3.5],[0.2857142984867096],[2],[6],[0.15523487676628825]
218,"[11782, 42075]","[12, 12]","[55, 55]","[30, 30]","[19.363636016845703, 19.363636016845703]","[4.583333492279053, 4.583333492279053]","[21, 21]","[17.0, 17.0]","[0.8779342770576477, 0.8779342770576477]","[24, 114]",...,"[14618.0, 1180.0]","[7895.0, 319.0]","[0.5400875806808472, 0.2703389823436737]","[1, 2]","[0.0833333358168602, 0.1666666716337204]","[4.0, 4.0]","[0.0833333358168602, 0.1666666716337204]","[11, 6]","[9, 3]","[0.025143670485964303, 0.057895144780092875]"
349,"[47755, 1890, 32650]","[9, 9, 9]","[157, 157, 157]","[85, 85, 85]","[27.25, 27.25, 27.25]","[17.44444465637207, 17.44444465637207, 17.4444...","[16, 16, 16]","[30.0, 30.0, 30.0]","[1.1009174585342407, 1.1009174585342407, 1.1

In [21]:
d[1553088]

'47209'